Importing Libraries needed to run script

In [1]:
import os
import numpy as np
import pandas as pd
import shutil
import csv

Removing existing results and making new directories to store new results

In [2]:
directories=os.listdir()
if "Automation" in directories:
    shutil.rmtree(os.getcwd()+"/Automation")
if "Log_Files" in directories:
    shutil.rmtree(os.getcwd()+"/Log_Files")    
if "Impl" in directories:
    shutil.rmtree(os.getcwd()+"/Impl") 
if "Program_Status.txt" in directories:
    os.remove(os.getcwd()+"/Program_Status.txt")
if "result.csv" in directories:
    os.remove(os.getcwd()+"/result.csv")    
os.mkdir("Automation")
os.mkdir("Impl")
os.mkdir("Log_Files")   

Generating a string containing the names of all the modules

In [3]:
modules_list = os.listdir("Modules")
files = [file for file in modules_list if ".v" or ".sv"  in file] 

In [5]:
files_string=""
for file in files:
    files_string=files_string+file+" "

Running the tcl_add script with files_string as an argument to add all the module files in a newly created vivado project

In [7]:
os.system("vivado -mode batch -source Script/tcl_add.tcl -tclargs {}".format(files_string))

0

Finding the top modules

In [8]:
top_modules = [mod for mod in files if mod[:3]=='top']

In [9]:
top_module_string=""
for file in top_modules:
    if ".v" in file:
        top_module_string=top_module_string+file[0:-2]+" "
    if ".sv" in file:
        top_module_string=top_module_string+file[0:-3]+" "        

Making an Impl folder with sub folders each having the name of a particular top_module to save its implementation results

In [11]:
for filename in top_modules:
    if ".v" in filename:
        os.mkdir("Impl/"+filename[:-2])
    if ".sv" in filename:
        os.mkdir("Impl/"+filename[:-3])  

Running the tcl_run script with top_module_string as an argument to run the synthesis and implementation of all the top_modules and store the results in Impl folder. Note that the results won't be generated for those files whose synthesis or implementation has failed. For them, a seperate Log_Files directory is generated which will have the runme.log having the reason of synthesis or implementation failure.

Also, note that the script continues on to synthesis the next module in case the current module fails.

In [12]:
os.system("vivado -mode batch -source Script/tcl_run.tcl -tclargs {}".format(top_module_string))

0

In [5]:
impl_dir = os.listdir("Impl")

Generating Result.csv file 
The approach is to search for certain text in the generated reports and extract out the numerical values needed.

In [7]:
DatapathDelay = []
logic_delay=[]
routing_delay=[]

file_name=[]

DynamicPower = []
static_power=[]
signal_power=[]
logic_power=[]
i_o_power=[]
total_power=[]


f7_muxes=[]
lut_as_memory=[]
lut_as_logic=[]
slice_lut=[]
lut1=[]
lut2=[]
lut3=[]
lut4=[]
lut5=[]
lut6=[]
lut7=[]

for mod in impl_dir:
    if ("timing.txt" not in os.listdir("Impl/{}".format(mod))):  # if implemented reports are not present, move to next iteration
        continue
    with open("Impl/{}/timing.txt".format(mod)) as f:
        lines = f.read()
        timing = lines.split("\n")
        try:
            timing_info = [line for line in timing if "Data Path Delay" in line]
            dpd = float(timing_info[0].split()[3][:-2])
            DatapathDelay.append(dpd)

            timing_info = [line for line in timing if "Data Path Delay" in line]
            dpd = float(timing_info[0].split()[5][:-2])
            logic_delay.append(dpd)

            timing_info = [line for line in timing if "Data Path Delay" in line]
            dpd = float(timing_info[0].split()[8][:-2])
            routing_delay.append(dpd)
        except:
            DatapathDelay.append("No Timing Paths Found")
            logic_delay.append("No Timing Paths Found")
            routing_delay.append("No Timing Paths Found")
    
    file_name.append(mod)    
    
    with open("Impl/{}/power.txt".format(mod)) as f:
        lines = f.read()
        power = lines.split("\n")
        
        try:
            power_info = [line for line in power if "Dynamic" in line]
            dpower = float(power_info[0].split("|")[2].strip())
            DynamicPower.append(dpower)
        except:
            DynamicPower.append("Not Generated")
            
        
        try:
            power_info = [line for line in power if "Static Power" in line]
            dpower = float(power_info[0].split("|")[2].strip())
            static_power.append(dpower)
        except:    
            static_power.append("Not Generated")

        try:    
            power_info = [line for line in power if "Signals" in line]
            #print(power_info)
            dpower = float(power_info[0].split("|")[2].strip())
            signal_power.append(dpower)        
        except:
            signal_power.append("Not Generated")
            
        try:    
            power_info = [line for line in power if "Slice Logic" in line]
            #print(power_info)
            dpower = float(power_info[0].split("|")[2].strip())
            logic_power.append(dpower)   

        except:
            logic_power.append("Not Generated") 
        try:    
            power_info = [line for line in power if "I/O" in line]
            #print(power_info)
            dpower = float(power_info[0].split("|")[2].strip())
            i_o_power.append(dpower) 

        except:    
            i_o_power.append("Not Generated") 
        try:    
            power_info = [line for line in power if "Total On-Chip Power" in line]
            #print(power_info)
            dpower = float(power_info[0].split("|")[2].strip())
            total_power.append(dpower) 
        except:
            total_power.append("Not Generated") 
            
    with open("Impl/{}/utilization.txt".format(mod)) as f:
        lines = f.read()
        uti = lines.split("\n")
        
        try:
            uti_info = [line for line in uti if "F7 Muxes" in line]
            util = int(uti_info[0].split("|")[2].strip())
            f7_muxes.append(util)
        except:
            f7_muxes.append(0)
        
        uti_info = [line for line in uti if "LUT as Memory" in line]
        util = int(uti_info[0].split("|")[2].strip())
        lut_as_memory.append(util)
        
        uti_info = [line for line in uti if "LUT as Logic" in line]
        util = int(uti_info[0].split("|")[2].strip())
        lut_as_logic.append(util)
        
        uti_info = [line for line in uti if "Slice LUTs" in line]
        util = int(uti_info[0].split("|")[2].strip())
        slice_lut.append(util)
        
        uti_info = [line for line in uti if "| LUT1" in line]
        try:
            util = int(uti_info[0].split("|")[2].strip())
            lut1.append(util)
        except:
            lut1.append(0)
            
        uti_info = [line for line in uti if "| LUT2" in line]
        try:
            util = int(uti_info[0].split("|")[2].strip())
            lut2.append(util)
        except:
            lut2.append(0)
            
            
        uti_info = [line for line in uti if "| LUT3" in line]
        try:
            util = int(uti_info[0].split("|")[2].strip())
            lut3.append(util)
        except:
            lut3.append(0)
            
            
        uti_info = [line for line in uti if "| LUT4" in line]
        try:
            util = int(uti_info[0].split("|")[2].strip())
            lut4.append(util)
        except:
            lut4.append(0) 
            
        uti_info = [line for line in uti if "| LUT5" in line]
        try:
            util = int(uti_info[0].split("|")[2].strip())
            lut5.append(util)
        except:
            lut5.append(0)
           
        uti_info = [line for line in uti if "| LUT6" in line]
        try:
            util = int(uti_info[0].split("|")[2].strip())
            lut6.append(util)
        except:
            lut6.append(0)        
            
        uti_info = [line for line in uti if "| LUT7" in line]
        try:
            util = int(uti_info[0].split("|")[2].strip())
            lut7.append(util)
        except:
            lut7.append(0)            

In [8]:
title_list=["Design Name","Datapath Delay","Logic Delay","Routing Delay","Dynamic Power","Static Power",
"Signal Power","Logic Power","IO Power","Total Power","F7 Muxes","LUT as Memory","LUT as Logic","Slice LUTs",
  "LUT1","LUT2","LUT3","LUT4","LUT5","LUT6","LUT7"]         

result = [file_name,DatapathDelay,logic_delay,routing_delay,DynamicPower,static_power,signal_power,logic_power,i_o_power,total_power,f7_muxes,lut_as_memory,lut_as_logic,slice_lut
,lut1,lut2,lut3,lut4,lut5,lut6,lut7]

resultT = np.array(result).T
df = pd.DataFrame(resultT)
df.columns = title_list

In [9]:
df.to_csv("result.csv")